Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [4]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
filtered = (ratings.groupby('userId')
            .agg({'movieId':'count', 'timestamp':['min','max']})
            .reset_index()
           )
filtered['LTV'] = filtered['timestamp']['max']-filtered['timestamp']['min']

filtered[filtered['movieId']['count'] > 100][['userId','LTV']].head()

,userId,LTV
,,
3,4,203560
7,8,85187
14,15,471393496
16,17,8053
18,19,5282


Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [7]:
import pandas as pd

rzd = pd.DataFrame({ 'client_id': [111, 112, 113, 114, 115],'rzd_revenue': [1093, 2810, 10283, 5774, 981]})
auto = pd.DataFrame({'client_id': [113, 114, 115, 116, 117],'auto_revenue': [57483, 83, 912, 4834, 98]})
air = pd.DataFrame({'client_id': [115, 116, 117, 118],'air_revenue': [81, 4, 13, 173]})


client_base = pd.DataFrame({'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
                            'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                                        'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']})

from typing import List
from functools import reduce

def merge_dfs(dfs: List[pd.DataFrame], **kwargs) -> pd.DataFrame:
    return reduce(lambda res, df: res.merge(df, **kwargs), dfs)

merged_data = merge_dfs([client_base, rzd, auto, air], on='client_id', how='outer')

merged_data

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? 
Ведь это просто числа, которые сами по себе мало что дают.

In [8]:
# исходные данные
user_geo = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 33, 77, 66, 88, 99],
        'location': 
        [('54.37825002707541', '26.938688037917018'), ('54.37825002707541', '26.938792979344726'), 
         ('54.378219014033675', '26.938899010419846'), ('54.37809596769512', '26.939004035666585'), 
         ('54.3779119849205', '26.939267981797457'), ('54.377789022400975', '26.939373007044196'),
         ('54.377081003040075', '26.939374012872577'),('54.37695804052055', '26.939268987625837')
        ]
    }
)
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

# подготовка таблиц
visits_pivot = visits.pivot_table(index='user_id', columns='source', values='user_id', 
                                  aggfunc='size', fill_value=0).reset_index()
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0).reset_index()
users = visits_pivot.merge(purchases_pivot, how='outer').merge(user_geo, how='left')
users

,user_id,ad,email,google,yandex,Авто,Дача,Спорт,location
0,11,1.0,0.0,1.0,0.0,0.0,0.0,2.0,"(54.37825002707541, 26.938688037917018)"
1,22,0.0,0.0,0.0,1.0,1.0,0.0,0.0,"(54.37825002707541, 26.938792979344726)"
2,55,0.0,1.0,0.0,0.0,0.0,1.0,0.0,"(54.378219014033675, 26.938899010419846)"
3,77,1.0,0.0,0.0,0.0,NaN,NaN,NaN,"(54.3779119849205, 26.939267981797457)"
4,99,NaN,NaN,NaN,NaN,1.0,0.0,0.0,"(54.37695804052055, 26.939268987625837)"


У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

>Мы знаем координаты пользователя в момент совершения покупки или просмотров, преобразуем их в адрес, например

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

>Смотря в каком временном отрезки учитываются эти координаты, пользователь мог в первых координат быть в одном городе, а когда координаты снова появились мог быть в другой городе. Так же удаленность координат может означать одновременное использование учетки на нескольких устройствах - но если использовать ID сессии - проблема решается.

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

>1) полный адрес с указанием до улицы, поэтому в таблицу покупок можно добавить адрес для последующей детализации

>2) часовые пояса, чтобы знать, когда клиент активен в течении дня

>3) расстояние до ближайших интересующих нас объектов

>4) метаданные фото, т.е. найти фотографии пользователя для последующего распознания (рекламные бренды и т.п.) или проанализировать фото рядом с местом и т.д.

>5) можно взять маскимально повторяющиеся координаты как самое популярное "место" визита приложения.

>6) характер передвижения и транспорт  и т.д.